In [1]:
import numpy as np 
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

import warnings
warnings.simplefilter("ignore")
import yfinance as yf

from plotly.subplots import make_subplots
from ta.trend import MACD
from ta.momentum import RSIIndicator
from ta.momentum import StochasticOscillator
import os
import pandas_ta as ta
import sys

# Relative Strength Index
The RSI is used to determine if a security is overbought or oversold. With them you can take advantage of potential changes in trend. The 2 most commonly used oscillators are the RSI and Stochastic RSI.

The RSI focuses on the deviation of upward and downward averages with values between 0 and 100. The RSI normally uses 9, 14 or 25 sessions which means it is used mainly as a short term analysis tool. A 14 session period is the most commonly used. When used with 9 sessions 0 to 20 is oversold and 80 to 100 is overbought. With 14 values over 70 are considered overbought and those below 30 oversold. When using 25 over 65 are considered overbought and those below 35 oversold.

This indicator is most commonly used with other indicators.
# Bollinger Bands
Bollinger Bands plot 2 lines using a moving average and the standard deviation defines how far apart the lines are. They also are used to define if prices are to high or low. When bands tighten it is believed a sharp price move in some direction. Prices tend to bounce off of the bands which provides potential market actions.

A strong trend should be noted if the price moves outside the band. If prices go over the resistance line it is in overbought territory and if it breaks through support it is a sign of an oversold position.

You normally use 20 sessions when using them.

High Band (Resistance) = Simple Moving Average + 2 * Standard Deviation
Low Band (Support) = Simple Moving Average - 2 * Standard Deviation

# Download Stock Data
Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y.
Valid intervals: 1d.

# Plot RSI / Bollinger Bands

In [4]:
PATH = r"C:\Users\91909\Desktop\Stock_Data"

def save_to_csv_from_yahoo(ticker: str) -> bool:
    df = yf.download(ticker, period="5y", progress=False)
    os.makedirs(PATH, exist_ok=True)
    fn = os.path.join(PATH, f"{ticker}.csv")
    df.to_csv(fn)
    return True

def get_stock_df_from_csv(ticker: str) -> pd.DataFrame:
    fn = os.path.join(PATH, f"{ticker}.csv")
    df = pd.read_csv(fn, index_col=0, parse_dates=True)
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    return df
    
def add_bollinger_bands(df: pd.DataFrame, window: int = 20, num_std: int = 2):
    df["middle_band"] = df["Close"].rolling(window).mean()
    df["std"] = df["Close"].rolling(window).std()
    df["upper_band"] = df["middle_band"] + num_std * df["std"]
    df["lower_band"] = df["middle_band"] - num_std * df["std"]
    
def add_rsi(df: pd.DataFrame, window: int = 14):
    delta = df["Close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    df["RSI"] = 100 - (100 / (1 + rs))

def plot(ticker: str):
    save_to_csv_from_yahoo(ticker)
    df = get_stock_df_from_csv(ticker)
    add_rsi(df)
    add_bollinger_bands(df)
    
    fig = go.Figure()

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df["Open"],
        high=df["High"],
        low=df["Low"],
        close=df["Close"],
        name="Price"
    ))         
    # RSI line
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["RSI"],
        mode="lines",
        name="RSI (14)",
        line=dict(color="purple"),
        yaxis="y2"
    ))

    # RSI overbought/oversold lines
    fig.update_layout(
        shapes=[
            dict(type="line", xref="paper", x0=0, x1=1, yref="y2", y0=70, y1=70,
                 line=dict(color="red", dash="dash")),
            dict(type="line", xref="paper", x0=0, x1=1, yref="y2", y0=30, y1=30,
                 line=dict(color="green", dash="dash"))
        ]
    )

    # Bollinger Bands
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["upper_band"],
        name="Upper Band",
        line=dict(color="rgba(255,0,0,0.7)", width=1)
    ))

    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["middle_band"],
        name="Middle Band",
        line=dict(color="rgba(0,0,255,0.5)", width=1)
    ))

    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["lower_band"],
        name="Lower Band",
        line=dict(color="rgba(0,255,0,0.7)", width=1)
    ))


    
    # Layout configuration
    fig.update_layout(
        title=f"{ticker} — 5y RSI Indicator",
        template="plotly_white",
        height=800,
        width=1100,
        showlegend=True,
        xaxis=dict(
            title="Date",
            rangeslider_visible=False,
            rangebreaks=[dict(bounds=["sat", "mon"])],
            rangeselector=dict(
                buttons=[
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ]
            )
        ),
        yaxis=dict(
            title="Price (INR)",
            domain=[0.35, 1]
        ),
        yaxis2=dict(
            title="RSI",
            domain=[0, 0.3],
            range=[0, 100],
            showgrid=True
        )
    )

    fig.show()

# Example usage
plot("NATCOPHARM.NS") 

YF.download() has changed argument auto_adjust default to True
